# Introduction: what problem are lenses designed to solve?

Let's say we have a collection of nested data types where we frequently want to modify the inner fields

In [1]:
import Data.Time
import Data.Map.Strict (Map)
import qualified Data.Map.Strict as M

In [2]:
data Account = Account
  { _name :: String
  , _sessions :: Map SessionID Session
  , _permissions :: Permissions
  }
  deriving Show

newtype SessionID = SessionID Int
  deriving (Show, Eq, Ord)

data Session = Session
  { _sessionStarted :: UTCTime
  , _lastActive  :: UTCTime
  }
  deriving Show

data Permissions = Permissions
  { _canEditPages :: Bool
  , _canDeletePages :: Bool
  , _canBanUsers :: Bool
  }
  deriving Show
  
homer = Account "Homer Simpson" M.empty (Permissions True False False)
print homer

Account {_name = "Homer Simpson", _sessions = fromList [], _permissions = Permissions {_canEditPages = True, _canDeletePages = False, _canBanUsers = False}}

Using Haskell's record syntax it's not hard to set the name field to something new

In [3]:
-- we can use record syntax to set the name
updateName :: String -> Account -> Account
updateName newName account = account { _name = newName }

print $ updateName "Homer J. Simpson" homer

Account {_name = "Homer J. Simpson", _sessions = fromList [], _permissions = Permissions {_canEditPages = True, _canDeletePages = False, _canBanUsers = False}}

In [4]:
import Data.Maybe (fromJust)
import Data.Time.Format.ISO8601 (iso8601ParseM)

newSession :: Session
newSession = Session (fromJust $ iso8601ParseM "2020-12-04T11:19:04Z") (fromJust $ iso8601ParseM "2020-12-04T12:47:32Z")

newSessionId :: SessionID
newSessionId = SessionID 4123

In [5]:
-- we can also use record syntax to modify
addSession :: SessionID -> Session -> Account -> Account
addSession sessionId session account = account { _sessions = M.insert sessionId session (_sessions account) }

print $ addSession newSessionId newSession homer

Account {_name = "Homer Simpson", _sessions = fromList [(SessionID 4123,Session {_sessionStarted = 2020-12-04 11:19:04 UTC, _lastActive = 2020-12-04 12:47:32 UTC})], _permissions = Permissions {_canEditPages = True, _canDeletePages = False, _canBanUsers = False}}

If we want to edit something that's nested things start getting a little hairy

In [6]:
toggleCanEditPages :: Account -> Account
toggleCanEditPages account = 
  account
    { _permissions = (_permissions account) 
      { _canEditPages = not (_canEditPages (_permissions account))
      }
    }
    
print $ toggleCanEditPages homer

Account {_name = "Homer Simpson", _sessions = fromList [], _permissions = Permissions {_canEditPages = False, _canDeletePages = False, _canBanUsers = False}}

## Lenses: composable record accessors!

In [7]:
{-# LANGUAGE TemplateHaskell #-}
import Control.Lens

makeLenses ''Account
makeLenses ''Session
makeLenses ''Permissions

-- generates eg. name :: Lens' Account Text

In [8]:
updateNameBetter :: String -> Account -> Account
updateNameBetter = set name

In [9]:
addSessionBetter :: SessionID -> Session -> Account -> Account
addSessionBetter sessionId session = over sessions (M.insert sessionId session)

In [10]:
-- lenses can compose!
toggleCanEditPagesBetter :: Account -> Account
toggleCanEditPagesBetter = over (permissions . canEditPages) not

### Like record accessors, we can also use lenses to view

In [11]:
print $ homer ^. permissions . canBanUsers

False

## The `Lens` tab, explained, one step at a time

The general shape of (not-type-changing) lenses and related structures in the `lens` library is:

```haskell
type Optic' s a = forall f. Constraint f => (a -> f a) -> s -> f s
```

For some constraint. The particular constraint for lenses proper is `Functor`. 

Note that if we have two such optics, we can compose them with (Prelude..)!

## Getters: basically functions

The simplest thing that record accessors do that `lens` wants to replicate is access data. We want something of the lens shape that works like a function! A bit of Church encoding magic gives us:

```haskell
type Getter1 s a = s -> a
-- is isomorphic to...
type Getter2 s a = forall r. (a -> r) -> s -> r
-- is isomorphic to...
type Getter3 s a = forall r. (a -> Const r a) -> s -> Const r s
-- is isomorphic to...
type Getter s a = forall f. (Functor f, Contravariant f) => (a -> f a) -> s -> f s
```

```haskell
type Getter s a = forall f. (Functor f, Contravariant f) => (a -> f a) -> s -> f s

to :: (s -> a) -> Getter s a
to f k s = contramap f (k (f s))

view :: Getter s a -> s -> a
view getter s = runConst (getter Const s)

(^.) :: s -> Getter s a -> a

views :: Getter s a -> (a -> r) -> s -> a

use :: MonadState s m => Getter s a -> m a

uses :: MonadState s m => Getter s a -> (a -> b) -> m b
```

## Setters: when you want to make changes

The next thing that lens wants to cover that we got from record accessors is setting. It turns out to be a little nicer if we think about _modifying_ rather than setting.

```haskell
type Setter1 s a = (a -> a) -> s -> s
-- is isomorphic to...
type Setter2 s a = (a -> Identity a) -> s -> Identity s
-- is isomorphic to...
type Setter' s a = forall f. Settable f => (a -> f a) -> s -> f s
```

`Settable` is a typeclass that lens defines that's basically "is isomorphic to `Identity`"

```haskell
class Applicative f => Settable f where
  untainted :: f a -> a

instance Settable Identity where
  untainted = runIdentity

type Setter' s a = forall f. Settable f => (a -> f a) -> s -> f s

setting :: ((a -> a) -> s -> s) -> Setter' s a
setting f k s = pure (f (untainted . k) s)

over :: Setter' s a -> (a -> a) -> s -> s
over setter f s = runIdentity (setter (Identity . f) s)

set :: Setter' s a -> a -> s -> s
set setter a = over setter (const a)

(%~) :: Setter' s a -> (a -> a) -> s -> s

(.~) :: Setter' s a -> a -> s -> s

(%=) :: MonadState s a => Setter' s a -> (a -> a) -> m ()

(.=) :: MonadState s a => Setter' s a -> a -> m ()
```

### Setter laws
```
over setter id = id
over setter f . over setter g = over setter (f . g)
```

These are essentially the functor laws!

### Type changing setters

We can actually generalize setters so that we can change the types!

```haskell
type Setter s t a b = forall f. Settable f => (a -> f b) -> s -> f t

setting :: ((a -> b) -> s -> t) -> Setter s t a b

mapped :: Functor f => Setter a b (f a) (f b)

over :: Setter s t a b -> (a -> b) -> s -> t
```

## Lenses: putting it all together

We want to end up with something that we can use as both a Getter and as a Setter. This means that we need the maximum constraint that is both a superconstraint to (Contravariant f, Functor f) and Settable f. This happens to be Functor. We'll keep the type-changing behaviour.

```haskell
type Lens s t a b = forall f. Functor f => (a -> f b) -> s -> f t

lens :: (s -> a) -> (s -> b -> t) -> Lens s t a b
lens get set k s = fmap (\b -> set s b) (k (get s))

(<%~) :: Lens s t a b -> (a -> b) -> s -> (a, t)
(<%~) l f s = l (\a -> (a, f a)) s
```

## Next time...

* Folds and Traversals
* Plated